In [1]:
import tensorflow as tf
from data_scripts.LevelDataset import LevelDataset

c:\Users\asus\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\asus\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\asus\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
dataset_path = "train_datasets/test_run_200/test_run_200.tfrecords"
dataset = LevelDataset(dataset_path = dataset_path, batch_size = 1)
dataset.load_dataset()
iter_data = dataset.get_dataset()

Image: Tensor("Cast:0", shape=(None, None, None), dtype=float32)
*****Dataset:**** <MapDataset element_spec=(TensorSpec(shape=(None, None, None), dtype=tf.float32, name=None), {'block_amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'level_height': TensorSpec(shape=(), dtype=tf.float32, name=None), 'level_width': TensorSpec(shape=(), dtype=tf.float32, name=None), 'pig_amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'pixel_height': TensorSpec(shape=(), dtype=tf.int64, name=None), 'pixel_width': TensorSpec(shape=(), dtype=tf.int64, name=None), 'platform_amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'special_block_amount': TensorSpec(shape=(), dtype=tf.int64, name=None)})>


In [3]:
for image_batch, data in iter_data:
    # print(data)
    print(image_batch.shape)
    print(data.keys())
    original = image_batch[0]
    break

(1, 128, 128, 5)
dict_keys(['block_amount', 'level_height', 'level_width', 'pig_amount', 'pixel_height', 'pixel_width', 'platform_amount', 'special_block_amount'])


In [4]:
dataset_path = "train_datasets/modified_test_run_200/modified_test_run_200.tfrecords"
modified_dataset = LevelDataset(dataset_path = dataset_path, batch_size = 1)
modified_dataset.load_dataset()
modified_iter_data = modified_dataset.get_dataset()

Image: Tensor("Cast:0", shape=(None, None, None), dtype=float32)
*****Dataset:**** <MapDataset element_spec=(TensorSpec(shape=(None, None, None), dtype=tf.float32, name=None), {'block_amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'level_height': TensorSpec(shape=(), dtype=tf.float32, name=None), 'level_width': TensorSpec(shape=(), dtype=tf.float32, name=None), 'pig_amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'pixel_height': TensorSpec(shape=(), dtype=tf.int64, name=None), 'pixel_width': TensorSpec(shape=(), dtype=tf.int64, name=None), 'platform_amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'special_block_amount': TensorSpec(shape=(), dtype=tf.int64, name=None)})>


In [5]:
for image_batch, data in modified_iter_data:
    # print(data)
    print(image_batch.shape)
    modified = image_batch[0]
    break

(1, 128, 128, 5)


In [6]:
import numpy as np
mask = tf.not_equal(original, modified)
indices = tf.where(mask)

# Print the positions where the elements are different
print("Positions where elements are different:")
print(np.where(mask))

num_differences = np.sum(mask)

# Print the number of differences
print("Number of differences:", num_differences)

Positions where elements are different:
(array([122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122,
       122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122,
       122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122,
       122, 122, 122, 122, 122, 122, 122, 122, 122, 123, 123, 123, 123,
       123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123,
       123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123,
       123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123,
       123, 123, 123, 123, 123, 124, 124, 124, 124, 124, 124, 124, 124,
       124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124,
       124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124,
       124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124,
       124], dtype=int64), array([27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35,
       35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42

In [7]:
for idx in indices:
    row_idx, col_idx, channel_idx = idx[0], idx[1], idx[2]
    original_value = tf.gather_nd(original, [idx])
    modified_value = tf.gather_nd(modified, [idx])
    print(f"Difference at position {idx}: original={original_value.numpy()}, modified={modified_value.numpy()}")

Difference at position [122  27   0]: original=[-1.], modified=[1.]
Difference at position [122  27   3]: original=[1.], modified=[-1.]
Difference at position [122  28   0]: original=[-1.], modified=[1.]
Difference at position [122  28   3]: original=[1.], modified=[-1.]
Difference at position [122  29   0]: original=[-1.], modified=[1.]
Difference at position [122  29   3]: original=[1.], modified=[-1.]
Difference at position [122  30   0]: original=[-1.], modified=[1.]
Difference at position [122  30   3]: original=[1.], modified=[-1.]
Difference at position [122  31   0]: original=[-1.], modified=[1.]
Difference at position [122  31   3]: original=[1.], modified=[-1.]
Difference at position [122  32   0]: original=[-1.], modified=[1.]
Difference at position [122  32   3]: original=[1.], modified=[-1.]
Difference at position [122  33   0]: original=[-1.], modified=[1.]
Difference at position [122  33   3]: original=[1.], modified=[-1.]
Difference at position [122  34   0]: original=[

In [ ]:
count = 0
for idx in indices:
    count += 1

    row_idx, col_idx, channel_idx = idx[0], idx[1], idx[2]
    original_value = tf.gather_nd(original, [idx])  # Get original value
    modified = tf.tensor_scatter_nd_update(
        modified, [[row_idx, col_idx, channel_idx]], original_value  # Update modified tensor
    )



print("Modified tensor with original values rewritten:")
print(modified)

In [8]:
from converter.gan_processing.DecodingFunctions import DecodingFunctions

# functions to move the output from [-1, 1] to [0, 1] range
decoding_functions = DecodingFunctions(threshold_callback = lambda: 0.5)
decoding_functions.set_rescaling(rescaling = tf.keras.layers.Rescaling)
decoding_functions.update_rescale_values(max_value = 1, shift_value = 1)
rescale_function = decoding_functions.rescale

# function to flatten the gan output to an image with 1 channel
decoding_function = decoding_functions.argmax_multilayer_decoding_with_air

In [9]:
import matplotlib.pyplot as plt

ref_img, _ = decoding_function(original)
# print(gan_outputs_reformatted[i].shape)

# save image trough matplotlib
plt.imshow(ref_img)
plt.savefig('original_level.png')
# clear plot
plt.clf()

ref_img, _ = decoding_function(modified)
# print(gan_outputs_reformatted[i].shape)

# save image trough matplotlib
plt.imshow(ref_img)
plt.savefig('modified1_level.png')
# clear plot
plt.clf()

<Figure size 432x288 with 0 Axes>

In [10]:
from converter.to_img_converter.MultiLayerStackDecoder import MultiLayerStackDecoder
from level.LevelVisualizer import LevelVisualizer
from level.LevelReader import LevelReader


def load_level_decoder():
    multilayer_stack_decoder = MultiLayerStackDecoder()
    multilayer_stack_decoder.round_to_next_int = True
    multilayer_stack_decoder.custom_kernel_scale = True
    multilayer_stack_decoder.minus_one_border = False
    multilayer_stack_decoder.combine_layers = True
    multilayer_stack_decoder.negative_air_value = -1
    multilayer_stack_decoder.cutoff_point = 0.5
    multilayer_stack_decoder.display_decoding = False
    return multilayer_stack_decoder

multilayer_stack_decoder = load_level_decoder()
level_visualizer = LevelVisualizer()
level_reader = LevelReader()

In [11]:
import os
level = multilayer_stack_decoder.decode(modified.numpy())
print("level", level)

fig, ax = plt.subplots(1, 1, dpi = 100)
level_visualizer.create_img_of_structure(
    level.get_used_elements(), use_grid = False, ax = ax, scaled = True
)
fig.savefig('modified1_decoded_level.png')
plt.clf()

# Save level to xml
level_xml = level_reader.create_level_from_structure(level.get_used_elements(), red_birds = True, move_to_ground = True)
level_reader.write_xml_file(level_xml, os.path.join("./", f'modified_level.xml'))

level Path: created, Blocks: 17 Pigs: 1 Platform: 0 Bird: 0


<Figure size 600x400 with 0 Axes>